In [38]:
import numpy as np
import pandas as pd
import os
import pickle
import json
from sklearn.externals import joblib
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('serv_rec.csv')

data['Service Recommended'] = data['Service Recommended'].replace({"PSP ":"PSP"})
data['Wired/Wireless'] = data['Wired/Wireless'].replace({"Wired ":"Wired", "wireless":"Wireless"})
data['Type of Product'] = data['Type of Product'].replace({"H Disk":"HDisk", "Hdisk":"HDisk", "Headphones":"Headphone"})
data['Antivirus Purchase (yrs)'] = data['Antivirus Purchase (yrs)'].replace({"NO":"0"})

data1 = data.drop(['Puchase Date','Last Update'], axis=1, inplace=True)

data['Faulty software'] = data['Faulty software'].astype('category')
data['Faulty hardware'] = data['Faulty hardware'].astype('category')
data['Type of Product'] = data['Type of Product'].astype('category')
data['Antivirus Purchase (yrs)'] = data['Antivirus Purchase (yrs)'].astype('int64')
data['Services Used'] = data['Services Used'].astype('category')
data['Wired/Wireless'] = data['Wired/Wireless'].astype('category')
data['Service Recommended'] = data['Service Recommended'].astype('category')
data['Type of Product'] = data['Type of Product'].astype('category') 

LE = preprocessing.LabelEncoder()

data['Faulty software'] = LE.fit_transform(data['Faulty software'])
data['Faulty hardware'] = LE.fit_transform(data['Faulty hardware'])
data['Services Used'] = LE.fit_transform(data['Services Used'])
data['Wired/Wireless'] = LE.fit_transform(data['Wired/Wireless'])
data['Service Recommended'] = LE.fit_transform(data['Service Recommended'])
data['Type of Product'] = LE.fit_transform(data['Type of Product'])

data.head()

bins = [-0.9,0.9,2,10]
labels = [1,2,3]
data['Purchase_binned'] = pd.cut(data['Purchase Interval'], bins=bins, labels=labels)
data.head()

bins = [-0.9,3,6,50]
labels = [1,2,3]
data['Warranty_binned'] = pd.cut(data['Warranty Expiry (months)'], bins=bins, labels=labels)

bins = [-1.9,-1,6,50]
labels = [1,2,3]
data['Backup_binned'] = pd.cut(data['Last BackUp'], bins=bins, labels=labels)

bins = [-1.9,-1,0.9,2,50]
labels = [1,2,3,4]
data['Antivirus_binned'] = pd.cut(data['Antivirus Purchase (yrs)'], bins=bins, labels=labels)

bins = [-0.9,6,50]
labels = [1,2]
data['Tickets_binned'] = pd.cut(data['Number of Tickets raised'], bins=bins, labels=labels)

bins = [-1.9,-0.9,1.9,50]
labels = [1,2,3]
data['Update_period_binned'] = pd.cut(data['Update Period'], bins=bins, labels=labels)
data2 = data.drop(['Purchase Interval','Warranty Expiry (months)','Last BackUp','Antivirus Purchase (yrs)','Number of Tickets raised','Update Period'], axis=1, inplace=True) 
data.head()
# type(data)
# data.columns
# 0,1,2,4,

,Type of Product,Faulty software,Faulty hardware,Services Used,Wired/Wireless,Service Recommended,Purchase_binned,Warranty_binned,Backup_binned,Antivirus_binned,Tickets_binned,Update_period_binned
0,0,2,1,2,1,5,3,2,2,4,1,1
1,1,0,3,1,2,1,2,1,1,1,1,1
2,4,2,0,1,0,8,1,1,2,3,1,2
3,2,0,1,2,1,3,2,3,1,1,2,1
4,4,2,1,2,0,3,3,1,2,3,2,2


In [43]:
x = data
for col in [4,2,1,0]:
    onehotencoder = OneHotEncoder(categorical_features = [col])
    x = onehotencoder.fit_transform(x).toarray()
# onehotencoder_1 = OneHotEncoder(categorical_features = [0])
# x = onehotencoder_1.fit_transform(x).toarray()
x

array([[0., 1., 0., ..., 4., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [0., 1., 0., ..., 3., 1., 2.],
       ...,
       [0., 1., 0., ..., 4., 1., 2.],
       [0., 1., 0., ..., 3., 1., 3.],
       [0., 1., 0., ..., 4., 1., 2.]])

In [45]:
df = pd.DataFrame(x)
df.shape

(104, 17)

In [29]:
one_hot = pd.get_dummies(data['Type of Product'])
# Drop column B as it is now encoded
data = data.drop('Type of Product',axis = 1)
# Join the encoded df
data = data.join(one_hot)

one_hot = pd.get_dummies(data['Type of Product'])
# Drop column B as it is now encoded
data = data.drop('Type of Product',axis = 1)
# Join the encoded df
data = data.join(one_hot)
data

one_hot = pd.get_dummies(data['Faulty software'])
# Drop column B as it is now encoded
data = data.drop('Faulty software',axis = 1)
# Join the encoded df
data = data.join(one_hot)
data




def hot_encode(df):
    obj_df = df.select_dtypes(include=['object'])
    return pd.get_dummies(df, columns=obj_df.columns).values
ohe = hot_encode(data)
ohe.shape

(104, 12)